# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

mongoimport --db uk_food --collection establishments --file /home/mo/Documents/UNC/homeWork/nosql-challenge/Starter_Code/Resources/establishments.json/establishments.json --jsonArray


In [86]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [87]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [88]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'gardenDB', 'local', 'met', 'uk_food']


In [89]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [90]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [91]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('647facb5d8b9cf3891da01e8'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,

In [92]:
# assign the collection to a variable
establishments = db['establishments']
establishments

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [93]:
# Create a dictionary for the new restaurant data
new_restrunt_data = {
 'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'NewRatingPending': False,
 'RatingValue': '5',
 }


In [94]:
# Insert the new restaurant into the collection
data_query = db.establishments
data_query.insert_one(new_restrunt_data)

In [95]:
# Check that the new restaurant was inserted
check = data_query.find_one({
 'BusinessName': 'Penang Flavours'})
pprint(check)


{'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'NewRatingPending': False,
 'RatingValue': 5,
 '_id': ObjectId('6480e3cbb24878e20bfd0959'),
 'geocode': {'latitude': None, 'longitude': None}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [96]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessType': 1  , 'BusinessTypeID': 1 , '_id' : 0}

BusinessTypeID = data_query.find_one(query,fields)
pprint(BusinessTypeID)
# for result in BusinessTypeID:
#     pprint(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [97]:
# Update the new restaurant with the correct BusinessTypeID
query = {'BusinessTypeID' : 1}
update = {'$set': {'BusinessTypeID': ''}}
result = data_query.update_one(query,update)



In [98]:
# Confirm that the new restaurant was updated
data_query.find_one({
 'BusinessName': 'Penang Flavours'})


{'_id': ObjectId('6480e3cbb24878e20bfd0959'),
 'AddressLine3': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'NewRatingPending': False,
 'RatingValue': 5,
 'geocode': {'longitude': None, 'latitude': None}}

In [99]:
# we can also use result.modified_count
pprint(result.modified_count)

1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [100]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName" : "Dover"}
count_local = data_query.count_documents(query)
print(f" the number of documents tha has LocalAuthorityName as Dover is : {count_local}")

 the number of documents tha has LocalAuthorityName as Dover is : 0


In [101]:
# Delete all documents where LocalAuthorityName is "Dover"

result = data_query.delete_many(query)
result



In [102]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName" : "Dover"}
print(data_query.find_one(query))

None


In [103]:
# Check that other documents remain with 'find_one'
data_query.find_one()

{'_id': ObjectId('647facb5d8b9cf3891da01e8'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:0

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [109]:
# Change the data type from String to Decimal for longitude and latitude
for document in data_query.find():
    # Get the current 'longitude' and 'latitude' values
    geocode = document.get('geocode', {})
    longitude = geocode.get('longitude')
    latitude = geocode.get('latitude')

   

    # If the value exists and is a string, convert it to float
    if longitude and isinstance(longitude, str):
        new_longitude = float(longitude)
    if latitude and isinstance(latitude, str):
        new_latitude = float(latitude)

    # Update the document
    data_query.update_one(
        {'_id': document['_id']},
        {'$set': {'geocode.longitude': new_longitude, 'geocode.latitude': new_latitude}},
    )
    


Use `update_many` to convert `RatingValue` to integer numbers.

In [105]:
#finding one document to find the type
document = establishments.find_one()

# Get the value of a field (e.g., longitude)
longitude = document.get('geocode', {}).get('longitude')
print(type(longitude))

<class 'float'>


In [106]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [107]:
def extract_rating(rating_dict):
    if isinstance(rating_dict, dict):
        for k, v in rating_dict.items():
            return extract_rating(v)
    elif isinstance(rating_dict, str):
        return rating_dict

# Find all the documents in the collection
for document in data_query.find():
    # Get the current 'RatingValue' value
    rating_value = document.get('RatingValue')

    # Extract the numeric string from the nested dictionary
    rating_str = extract_rating(rating_value)

    # If the value exists and is a digit, convert it to int and update the document
    if rating_str and rating_str.isdigit():
        new_rating_value = int(rating_str)

        # Update the document
        data_query.update_one(
            {'_id': document['_id']},
            {'$set': {'RatingValue': new_rating_value}},
        )

In [108]:
# Check that the coordinates and rating value are now numbers
#finding one document to find the type
document = establishments.find_one()

# Get the value of a field (e.g., longitude)
longitude = document.get('RatingValue', {})
print(type(longitude))


<class 'int'>
